In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import TensorBoard
import datetime
import pickle
print('Versão do TensorFlow: ', tf.__version__)

2024-06-23 16:54:56.422995: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Versão do TensorFlow:  2.16.1


## Test GPU

In [4]:
tf.config.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [7]:
import numpy as np
# Action
# 0 Nothing
# 1 Go Left 
# 2 Go Right
# 3 RoteteLeft
# 4 RotateRight
# 5 TurnOnEngine

# Gerar os dados conforme especificado


data_rotate_left = []
data_rotate_right = []
data_go_right = []
data_go_left = []

# Preencher as listas de dados conforme as condições especificadas
for x in range(900):
    for currentX in range(900):
        for angle in range(-180, 181):
            if -90 <= angle <= -5:
                data_rotate_left.append([x, currentX, angle, 3])  # RotateLeft
            elif 5 <= angle <= 90:
                data_rotate_right.append([x, currentX, angle, 4])  # RotateRight
            elif -5 < angle < 5:
                if currentX < x:
                    data_go_right.append([x, currentX, angle, 2])  # Go Right
                else:
                    data_go_left.append([x, currentX, angle, 1])  # Go Left
            # else:
            #     data_nothing.append([x, currentX, angle, 0])  # Nothing


# Determinar o tamanho mínimo entre as listas para balanceamento
min_size = min(len(data_rotate_left), len(data_rotate_right), len(data_go_right), len(data_go_left))

# Balancear as listas
balanced_data = (
    data_rotate_left[:min_size] +
    data_rotate_right[:min_size] +
    data_go_right[:min_size] +
    data_go_left[:min_size]
)

# Contar a quantidade de cada tipo
 
count_rotate_left = sum(1 for item in balanced_data if item[3] == 3)
count_rotate_right = sum(1 for item in balanced_data if item[3] == 4)
count_go_right = sum(1 for item in balanced_data if item[3] == 2)
count_go_left = sum(1 for item in balanced_data if item[3] == 1)

# Mostrar a quantidade de cada tipo

print("Quantidade de 'RotateLeft':", count_rotate_left)
print("Quantidade de 'RotateRight':", count_rotate_right)
print("Quantidade de 'Go Right':", count_go_right)
print("Quantidade de 'Go Left':", count_go_left)

balanced_data = np.array(balanced_data)
X = balanced_data[:, :3]  # Features: x, currentX e angle
y = balanced_data[:, 3]   # Labels: 0, 1, 2, 3 ou 4

len(balanced_data)

Quantidade de 'Nothing': 0
Quantidade de 'RotateLeft': 3640950
Quantidade de 'RotateRight': 3640950
Quantidade de 'Go Right': 3640950
Quantidade de 'Go Left': 3640950


14563800

In [8]:

# Dividir os dados em conjuntos de treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)



In [9]:

# Salvar os dados em um arquivo .pkl
with open('data.pkl', 'wb') as f:
    pickle.dump([X_train, y_train, X_test, y_test], f)


In [10]:
# Carregar os dados do arquivo .pkl
with open('./data.pkl', 'rb') as f:
    X_train, y_train, X_test, y_test = pickle.load(f)

In [11]:
%load_ext tensorboard
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

# Construir o modelo
model = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation='relu', input_shape=(3,)),  # Ajustar a entrada para 3 features
    tf.keras.layers.Dense(64, activation='relu'),  # Ajustar a entrada para 3 features
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(5, activation='softmax')  # 5 saídas: 0, 1, 2, 3, 4
])

# Compilar o modelo
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Treinar o modelo
model.fit(X_train, y_train, epochs=100, batch_size=32000, validation_split=0.2, callbacks=[tensorboard_callback])


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:85: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
2024-06-23 17:07:58.439646: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-06-23 17:07:58.439861: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-06-23 17:07:58.439934: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have bee

Epoch 1/100


I0000 00:00:1719162481.434022     739 service.cc:145] XLA service 0x7f19bc0060f0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1719162481.434075     739 service.cc:153]   StreamExecutor device (0): NVIDIA GeForce RTX 4070, Compute Capability 8.9
2024-06-23 17:08:01.472916: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-06-23 17:08:01.642399: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:465] Loaded cuDNN version 8906
I0000 00:00:1719162484.137730     787 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_588', 12 bytes spill stores, 12 bytes spill loads

I0000 00:00:1719162485.640770     787 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_210', 32 bytes spill stores, 40 bytes spill loads



 23/292 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.4179 - loss: 10.6424

I0000 00:00:1719162488.285431     739 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


283/292 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8651 - loss: 1.7551

I0000 00:00:1719162490.890749     948 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_210', 60 bytes spill stores, 60 bytes spill loads

I0000 00:00:1719162491.058323     953 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_588', 148 bytes spill stores, 148 bytes spill loads



292/292 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - accuracy: 0.8680 - loss: 1.7124

I0000 00:00:1719162498.298770    1126 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_39', 60 bytes spill stores, 60 bytes spill loads



292/292 ━━━━━━━━━━━━━━━━━━━━ 20s 40ms/step - accuracy: 0.8683 - loss: 1.7078 - val_accuracy: 0.9926 - val_loss: 0.0198
Epoch 2/100
292/292 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.9928 - loss: 0.0185 - val_accuracy: 0.9930 - val_loss: 0.0160
Epoch 3/100
292/292 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.9942 - loss: 0.0140 - val_accuracy: 0.9951 - val_loss: 0.0106
Epoch 4/100
292/292 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.9963 - loss: 0.0092 - val_accuracy: 0.9971 - val_loss: 0.0071
Epoch 5/100
292/292 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.9977 - loss: 0.0067 - val_accuracy: 0.9973 - val_loss: 0.0062
Epoch 6/100
292/292 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.9886 - loss: 0.0778 - val_accuracy: 0.9973 - val_loss: 0.0073
Epoch 7/100
292/292 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.9973 - loss: 0.0072 - val_accuracy: 0.9969 - val_loss: 0.0077
Epoch 8/100
292/292 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.9982 - loss: 0.0062 - val_accuracy: 0.99

In [12]:

# Avaliar o modelo
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Model Accuracy: {accuracy * 100:.2f}%')


91024/91024 ━━━━━━━━━━━━━━━━━━━━ 176s 2ms/step - accuracy: 1.0000 - loss: 4.8812e-04
Model Accuracy: 100.00%


In [13]:
 


targetX = 680
agentX = 340
angle = -50

predictions = model.predict(np.array([[targetX, agentX, angle]]))
# 0 Nothing
# 1 Go Left
# 2 Go Right
# 3 RoteteLeft
# 4 RotateRight
# 5 TurnOnEngine
class_names = ['Nothing', 'Go Left', 'Go Right',
               'RoteteLeft', 'RotateRight', 'TurnOnEngine']

score = tf.nn.softmax(predictions[0])
print(
    "This image most likely belongs to {} with a {:.2f} percent confidence."
    .format(class_names[np.argmax(score)], 100 * np.max(score))
)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 309ms/step
This image most likely belongs to RoteteLeft with a 40.46 percent confidence.


In [ ]:
%tensorboard

In [14]:
model.save("./models/model_saved.h5")
model.save(filepath="./models/model_saved.keras")


 

#### The command below is to copy from the docker container to local folder

In [ ]:
!docker cp cool_spence:/tf/models/. /mnt/d/git/victorbarbosa/src/assets/models/

### From here to you only MACOS to test 

In [21]:
import tensorflow as tf
import numpy as np
model = tf.keras.models.load_model("./models/model_saved.keras")

targetX = 680
agentX = 300
angle = -50

predictions = model.predict(np.array([[targetX, agentX, angle]]))
# 0 Nothing
# 1 Go Left
# 2 Go Right
# 3 RoteteLeft
# 4 RotateRight
# 5 TurnOnEngine
class_names = ['Nothing', 'Go Left', 'Go Right',
               'RoteteLeft', 'RotateRight', 'TurnOnEngine']

score = tf.nn.softmax(predictions[0])
print(
    "This image most likely belongs to {} with a {:.2f} percent confidence."
    .format(class_names[np.argmax(score)], 100 * np.max(score))
)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 154ms/step
This image most likely belongs to RoteteLeft with a 40.46 percent confidence.


In [19]:
# !pip install tensorflowjs
from tensorflow.keras.models import load_model
import tensorflow as tf

# Carregar o modelo do arquivo .h5
model = load_model('./models/model_saved.h5')

# Compile o modelo (você pode precisar ajustar a função de perda, otimizador e métricas conforme necessário)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Salvar o modelo no formato SavedModel
tf.saved_model.save(model, './models/saved_model/')

!tensorflowjs_converter --input_format=tf_saved_model --output_format=tfjs_graph_model './models/saved_model/' models/web_model

INFO:tensorflow:Assets written to: ./models/saved_model/assets


INFO:tensorflow:Assets written to: ./models/saved_model/assets


2024-06-23 17:22:17.213101: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-23 17:22:19.511719: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-06-23 17:22:19.515849: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-06-23 17:22:19.515908: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUM